
## Treatment Safety Network Study Using Strategus - PLP Study

#### Step 1 - Import the necessary libraries


In [ ]:
%%jupyter
# import libraries
library(rD2E)
library(Strategus)
library(dplyr)

#### Step 2 - Cohort definition loading

In [ ]:
%%jupyter
tarCohortId <- 12
outCohortId <- 11
cohorts_set <- c(tarCohortId, outCohortId)
cohortDefinitionSet <- rD2E::get_cohort_definition_set(cohorts_set)


#### Step 3 - Define the network study components

In [ ]:
%%jupyter
# If you are not restricting your study to a specific time window,
# please make these strings empty
studyStartDate <- '19001201' # YYYYMMDD
studyEndDate <- '20231231'   # YYYYMMDD


# PLP time-at-risks should try to use fixed-time TARs
plpTimeAtRisks <- tibble(
  riskWindowStart  = c(1),
  startAnchor = c("cohort start"),
  riskWindowEnd  = c(365),
  endAnchor = c("cohort start")
)

# Outcomes: The outcome for this study is cohort_id == outcomeCohortId
oList <- cohortDefinitionSet %>%
  filter(.data$cohortId == outCohortId) %>%
  mutate(outcomeCohortId = cohortId, outcomeCohortName = cohortName) %>%
  select(outcomeCohortId, outcomeCohortName) %>%
  mutate(cleanWindow = 365)

Create settings for modules involved in the study

In [ ]:
%%Jupyter
# CohortGeneratorModule --------------------------------------------------------
cgModuleSettingsCreator <- CohortGeneratorModule$new()
cohortDefinitionShared <- cgModuleSettingsCreator$createCohortSharedResourceSpecifications(cohortDefinitionSet)
cohortGeneratorModuleSpecifications <- cgModuleSettingsCreator$createModuleSpecifications(
  generateStats = TRUE
)

# PatientLevelPredictionModule -------------------------------------------------
plpModuleSettingsCreator <- PatientLevelPredictionModule$new()

modelSettings <- list(
  lassoLogisticRegression = PatientLevelPrediction::setLassoLogisticRegression()
)

In [ ]:
%%jupyter
modelDesignList <- list()
tcIds <- cohortDefinitionSet %>%
  filter(cohortId != 3) %>%
  pull(cohortId)

for (cohortId in tcIds) {
  for (j in seq_len(nrow(plpTimeAtRisks))) {
    for (k in seq_len(nrow(oList))) {
      priorOutcomeLookback <- 99999
      for (mSetting in modelSettings) {
        modelDesignList[[length(modelDesignList) + 1]] <- PatientLevelPrediction::createModelDesign(
          targetId = cohortId,
          outcomeId = oList$outcomeCohortId[k],
          restrictPlpDataSettings = PatientLevelPrediction::createRestrictPlpDataSettings(
            sampleSize = 1000000,
            studyStartDate = studyStartDate,
            studyEndDate = studyEndDate,
            firstExposureOnly = FALSE,
            washoutPeriod = 0
          ),
          populationSettings = PatientLevelPrediction::createStudyPopulationSettings(
            riskWindowStart = plpTimeAtRisks$riskWindowStart[j],
            startAnchor = plpTimeAtRisks$startAnchor[j],
            riskWindowEnd = plpTimeAtRisks$riskWindowEnd[j],
            endAnchor = plpTimeAtRisks$endAnchor[j],
            removeSubjectsWithPriorOutcome = TRUE,
            priorOutcomeLookback = priorOutcomeLookback,
            requireTimeAtRisk = FALSE,
            binary = TRUE,
            includeAllOutcomes = TRUE,
            firstExposureOnly = FALSE,
            washoutPeriod = 0,
            minTimeAtRisk = plpTimeAtRisks$riskWindowEnd[j] - plpTimeAtRisks$riskWindowStart[j],
            restrictTarToCohortEnd = FALSE
          ),
          covariateSettings = FeatureExtraction::createCovariateSettings(
            useDemographicsGender = TRUE,
            useDemographicsAgeGroup = TRUE,
            useConditionGroupEraLongTerm = TRUE,
            useDrugGroupEraLongTerm = TRUE,
            useVisitConceptCountLongTerm = TRUE
          ),
          preprocessSettings = PatientLevelPrediction::createPreprocessSettings(),
          modelSettings = mSetting
        )
      }
    }
  }
}

plpModuleSpecifications <- plpModuleSettingsCreator$createModuleSpecifications(
  modelDesignList = modelDesignList
)

Create the analysis specification object - including all the modules and configurations created above

In [ ]:
%%jupyter
# Create the analysis specifications ------------------------------------------
analysisSpecifications <- Strategus::createEmptyAnalysisSpecificiations() |>
  Strategus::addSharedResources(cohortDefinitionShared) |> 
  Strategus::addModuleSpecifications(cohortGeneratorModuleSpecifications) |>
  Strategus::addModuleSpecifications(plpModuleSpecifications)

#### Step 4 - Execute the Strategus study

In [ ]:
%%jupyter
# temporary workaround for running strategus on d2e server
executionSettings <- Strategus::createCdmExecutionSettings(
    workDatabaseSchema = "cdm_5pct_9a0f90a32250497d9483c981ef1e1e70",
    cdmDatabaseSchema = "cdm_5pct_9a0f90a32250497d9483c981ef1e1e70",
    cohortTableNames = CohortGenerator::getCohortTableNames(cohortTable = "cohort"),
    workFolder = "/tmp/<notebook_name>/work",
    resultsFolder = "/tmp/<notebook_name>/results",
    logFileName = "/tmp/<notebook_name>/strategus-log.txt",
    minCellCount = 5,
    maxCores = 8,
  )

In [ ]:
%%jupyter
study_name <- "treatment_safety_study"  # Unique study name
options <- create_options(upload_results=TRUE, study_id = study_name) # set a study_id with a unique id
options$studyId <- study_name
rD2E::run_strategus_flow(analysisSpecification = analysisSpecifications, options = options)